In [ ]:
!pip install -q -U "google-generativeai>=0.7.2"

In [ ]:
from google.colab import userdata
import google.generativeai as genai
genai.configure(api_key=userdata.get('GOOGLE_API_KEY')

# Set up a Model with Tools

In [ ]:
def enable_lights():
  """Turn on the lighting system."""
  print("LIGHTBOT: Lights turned on.")

def set_light_color(reg_hex: str):
  """Set the light color. Lights must be enabled for this to work."""
  print(f"LIGHTBOT: Lights set to {reg_hex}.")

def stop_lights(reg_hex: str):
  """Stop flashing lights"""
  print(f"LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=light_controls,
    system_instruction=instruction
)

chat = model.start_chat()


In [ ]:
from google.genrativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
  """Create a tool config with the specified function calling mode."""
  return content_types.to_tool_config(
      {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
  )

In [ ]:
tool_config = tool_config_from_mode("none")

response = chat.send_message("Hello light-bot, what can you do?", tool_config=tool_config)
print(response.text)

In [ ]:
response.parts[0]

In [ ]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)


In [ ]:
print(response.parts)

# Automatic function calling

In [ ]:
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("auto", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
res = auto_chat.send_message("It's awful dark in here...", took_config=tool_config)

In [ ]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
res = auto_chat.send_message("It's awful dark in here - make it Orange and then switch off lights", took_config=tool_config)

In [ ]:
res

In [ ]:
def set_light_values(brightness, color_temp):
  """Set the brightness and color temperature of a room light. (mock API).

  Arg:
      brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
      color_temp: Color temperature of the light fixture, which can be 'daylight', 'cool' or 'warm'

  Returns:
      A dictionary containing the set of brightness and color temperature
  """
  return {"brightness": brightness, "colorTemperature": color_temp}